# 神经网络

将对于数据的操作的各个模块或层次组织在一起，就是神经网络模型。`torch.nn`模块提供了所有的用于建立神经网络模型的模块的命名空间(隐藏层、输出层等)。所有的模型都是`nn.Moudle`的子类。一个神经网络模型可以包括许多的其他的模型，这样就可以建立很复杂的结构。

In [1]:
# 导入相应的包
import torch
import os
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# 获取可以执行的设备
print("计算运行设备:", device)

计算运行设备: cuda


# 建立模型
可以通过继承`nn.Module`类来实现模型的建立，然后在`__init__`函数里初始化建立模型。所有的Module子类都需要实现前向传播函数`forward`,来实现对输入数据的处理。

In [2]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork,self).__init__()
        self.flatten = nn.Flatten()   # 将图像扁平化
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
        )

    def forward(self, x):
        x = self.flatten(x)  # 获取每一batch的输入，并利用第一个模块扁平化
        logits = self.linear_relu_stack(x) # 放入第二个复杂的模型进行训练
        return logits
    
# 将模型放到对应的结构上
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


# 使用模型
为了使用模型，可以直接向模型传递输入数据，上面的模型最后将会输出一个10维的`tensor`结构，这些操作都是在后台进行的，无需显式的调用`forward`函数进行计算。因为这是一个多分类问题，将上述的10维的结构放入一个`softmax`分类器里面，就可以得到概率最大的那个类别，也就是最终的分类。

In [3]:
x = torch.rand(1, 28, 28, device=device)
print("x",x)
logits = model(x)
print("logits",logits)
pred_probab = nn.Softmax(dim=1)(logits)
print("pred_probab",pred_probab)
y_pred = pred_probab.argmax(1)
print("y_pred",y_pred)

x tensor([[[8.3332e-01, 6.4700e-01, 2.6904e-01, 9.4117e-01, 8.3041e-01,
          6.8950e-01, 7.5871e-01, 7.2565e-01, 3.7627e-01, 7.5096e-01,
          2.8476e-01, 5.8763e-01, 3.6398e-01, 6.8863e-01, 4.3805e-01,
          2.2661e-01, 6.4731e-01, 3.0593e-01, 7.2615e-01, 8.8779e-01,
          9.0504e-01, 4.8914e-01, 5.3835e-01, 3.1738e-01, 3.1175e-01,
          1.6134e-01, 2.9150e-01, 1.7263e-02],
         [7.9437e-01, 6.8342e-01, 3.2559e-01, 6.7508e-01, 6.9991e-01,
          6.6925e-01, 7.2385e-01, 8.4121e-02, 4.8750e-01, 5.2389e-01,
          2.9232e-01, 5.5563e-01, 4.7164e-01, 3.9739e-01, 5.1104e-01,
          5.8132e-03, 4.8609e-01, 4.9585e-01, 8.2917e-01, 8.7852e-02,
          2.8830e-01, 5.9997e-01, 1.5156e-01, 1.5843e-02, 4.6177e-01,
          3.5969e-01, 9.1840e-01, 2.1088e-01],
         [9.1114e-01, 4.1642e-04, 2.7588e-01, 4.5763e-02, 3.0718e-01,
          2.1817e-01, 6.5364e-01, 5.1978e-01, 2.8948e-01, 7.5754e-01,
          2.3738e-01, 9.6432e-01, 3.2489e-01, 2.4330e-01, 6.0595

由上面可以看出来预测的变化，经过分类器之后，可以得到概率最大的那个类别，也就是随即的类别。**但是这里其实并没有真正的训练，只是将随即的变量代入了模型，模型内部的所有的参数都是初始化来的，并不是学习来的。**，此处只是简单的建立了模型，模型的训练在后面。

# 模型层次详解

## nn.Flatten层
顾名思义，这是一个扁平化层，是将输入的图像，本来是三个通道每个通道的有`28*28`个像素的图片，现在将这个28*28个像素展开，即每个通道都展开，这样做的好处就是一次性可以将所有的像素输入进去。

## nn.Linear 层
这是一个将线性转换器应用到模型的一层，利用其自身存储好的权重(weight)和偏移值(bias),来
进行线性计算。

## nn.ReLU层
这是非线性的激活函数，用于创建复杂的输入与输出之间的映射。放在了线性模型的输入之后，目的是引入非线性，帮助神经网络模型来学习更加丰富多彩的现象。

## nn.Sequential 层
这是一个有序的模型容器。输入数据被放入这个序列中，依次按照序列的定义来对数据进行操作。可以使用Sequential容器来快捷的创立一个模型。

## nn.Softmax 层
模型的上一层输出的是一个logits tensor，是一个介于无穷小到无穷大之间的数字。这个数字我们没有办法拿来预测，因此需要将这些值按比例缩小到[0,1]之间，并且每一个类别的概率之和最终等于一，便可以利用概率的知识，来判断预测的类别。其中的`dim=1`表示的就是最终的值的和必须符合的维度。

In [4]:
# 模型层次详解测试
input_data = torch.rand(3,28,28)
print(input_data.size())

# 展开层
flatten = nn.Flatten()
flat_image = flatten(input_data)
print(flat_image.size())

# 线性层
linear = nn.Linear(28*28,20)
hidden = linear(flat_image)
print(hidden.size())

# ReLU层
hidden = nn.ReLU()(hidden)
print(hidden.size())

# Sequential 容器
seq_modules = nn.Sequential(
    flatten,
    linear,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
print(logits)

## softmax 层
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab)

torch.Size([3, 28, 28])
torch.Size([3, 784])
torch.Size([3, 20])
torch.Size([3, 20])
tensor([[-0.0143, -0.3681, -0.3948, -0.1156,  0.1930, -0.0073, -0.0252, -0.0910,
          0.2857,  0.1458],
        [-0.1536, -0.2145, -0.3593, -0.2224, -0.1062,  0.1601,  0.0647, -0.1182,
          0.2194,  0.1233],
        [-0.0449, -0.2724, -0.2270, -0.3281,  0.0094,  0.0651, -0.0648,  0.0307,
          0.2427,  0.1028]], grad_fn=<AddmmBackward0>)
tensor([[0.1003, 0.0704, 0.0686, 0.0907, 0.1235, 0.1011, 0.0993, 0.0929, 0.1355,
         0.1178],
        [0.0896, 0.0843, 0.0730, 0.0837, 0.0940, 0.1227, 0.1115, 0.0929, 0.1302,
         0.1182],
        [0.0989, 0.0788, 0.0825, 0.0745, 0.1045, 0.1104, 0.0970, 0.1067, 0.1319,
         0.1147]], grad_fn=<SoftmaxBackward0>)


# 模型参数
在模型的显式训练的过程中，是有很多的参数的。我们可以将这些参数打印出来，nn.Module会自动跟踪定义的所有的模型，并保存其参数值。可以使用模型的`parameters()`或者`named_parameters()`方法来获取模型的参数。

In [5]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0067, -0.0107, -0.0082,  ...,  0.0005,  0.0229,  0.0243],
        [-0.0280, -0.0118,  0.0119,  ...,  0.0350,  0.0310,  0.0104]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0112, 0.0276], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 2.1209e-02, -3.7238e-02,  1.8224e-02,  ...,  2.9277e-02,
         -2.1018e-02,  4.3991e-05],
        [-2.8605e-02,  5.1087e-03,  2.6520e-03,  ..., -2.8026e-02,
      

over